In [17]:
# 必要なライブラリのインストール
%pip install -U weaviate-client

Note: you may need to restart the kernel to use updated packages.


In [24]:
# クライアントの初期化
import json
import glob
import os
import tqdm
import base64
import weaviate

client = weaviate.Client(
    url = "http://" + os.environ.get("WEAVIATE_HOST"),
    # url = "http://weaviate:8080"
    # additional_headers = {
    #     "X-OpenAI-Api-Key": os.environ.get("OPENAI_APIKEY")
    # }
)

In [19]:
# スキーマ定義
class_obj = {
    "class": "Cifar10",
    "description": "Each example is a 32x32 color image, associated with a label from 10 classes.",
    "vectorizer": "img2vec-neural",
    "moduleConfig": {
        "img2vec-neural": {
            "imageFields": [
                "image"
            ]
        }
    },
    "properties": [
        {
            "dataType": [
                "blob"
            ],
            "description": "color image",
            "name": "image"
        },
        {
            "dataType": [
                "text"
            ],
            "description": "file name",
            "name": "fileName"
        }
    ],
}

# 初回のみcreate_classを実行します
client.schema.create_class(class_obj)

In [20]:
# スキーマ登録できているか確認
client.schema.get("Cifar10")

{'class': 'Cifar10',
 'description': 'Each example is a 32x32 color image, associated with a label from 10 classes.',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'img2vec-neural': {'imageFields': ['image']}},
 'multiTenancyConfig': {'enabled': False},
 'properties': [{'dataType': ['blob'],
   'description': 'color image',
   'indexFilterable': False,
   'indexSearchable': False,
   'moduleConfig': {'img2vec-neural': {}},
   'name': 'image'},
  {'dataType': ['text'],
   'description': 'file name',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'img2vec-neural': {}},
   'name': 'fileName',
   'tokenization': 'word'}],
 'replicationConfig': {'factor': 1},
 'shardingConfig': {'virtualPerPhysical': 128,
  'desiredCount': 1,
  'actualCount': 1,
  'desiredVirtualCount': 128,
  'actualVirtualCount': 128,
  'key': '_id',
  'strategy':

In [21]:
# スキーマの削除
# client.schema.delete_class("Cifar10")

In [22]:
def add_image_to_weaviate(image_path):
    # 画像をバイナリ形式で読み込む
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

    # Weaviateに追加するデータオブジェクトを作成
    data_object = {
        "image": encoded_image,
        "fileName": os.path.basename(image_path),
    }

    # データをWeaviateに追加
    client.data_object.create(data_object, "Cifar10")

In [26]:
# 画像を格納する
img_paths = glob.glob('img/dump/*.png')
print(f"image count: {len(img_paths)}")
for img_path in tqdm.tqdm(img_paths):
    add_image_to_weaviate(img_path)

image count: 102


100%|██████████| 102/102 [00:14<00:00,  7.10it/s]


In [12]:
# 類似度の高い画像を検索する
near_img_path = "img/test/39_dog.png"
certainty = 0.90

result = (
  client.query
  .get("Cifar10", ["fileName"])
  .with_near_image({"image": near_img_path, "certainty": certainty})
  .do()
)

print(json.dumps(result, indent=2))

{
  "data": {
    "Get": {
      "Cifar10": [
        {
          "fileName": "83_dog.png"
        },
        {
          "fileName": "17_cat.png"
        }
      ]
    }
  }
}


In [11]:
# 類似度の高い画像を検索する
near_img_path = "test/28_truck.png"
certainty = 0.88

result = (
  client.query
  .get("Cifar10", ["fileName"])
  .with_near_image({"image": near_img_path, "certainty": certainty})
  .do()
)

print(json.dumps(result, indent=2))

{
  "data": {
    "Get": {
      "Cifar10": [
        {
          "fileName": "16_truck.png"
        },
        {
          "fileName": "1_truck.png"
        },
        {
          "fileName": "14_truck.png"
        },
        {
          "fileName": "32_automobile.png"
        },
        {
          "fileName": "31_truck.png"
        },
        {
          "fileName": "44_automobile.png"
        },
        {
          "fileName": "76_truck.png"
        },
        {
          "fileName": "71_truck.png"
        },
        {
          "fileName": "60_automobile.png"
        },
        {
          "fileName": "46_automobile.png"
        },
        {
          "fileName": "65_automobile.png"
        },
        {
          "fileName": "64_automobile.png"
        },
        {
          "fileName": "61_automobile.png"
        },
        {
          "fileName": "45_automobile.png"
        }
      ]
    }
  }
}
